# **Cloud Computing Assignement**

**Introduction** : This code is meant for the automation for the set up and the deployement of our distributed computing enviroment that consists of various elements including DynamoDB tables, SQS queues, and EC2 instances .We are going to deploy a web application where the user can retreive valuable informations about the air quality  according to  various criteria(location, date, range .....),The script is loaded with error handlers for easier debugging.

**Attention !!** : You can find further information about the execution in the file "Instructions.pdf".

**Package Installation** 

In [1]:
#pip install boto3
#pip install subprocess
#pip install paramiko
#pip install time 

**Libraries importation**

In [2]:
import boto3
import subprocess
import time
import paramiko

**Defining the necessary functions**

We will start by defining a function for the creation of a DynamoDB table meant for the storage of the air quality data

In [3]:
def create_DB(dynamoDB):
        table = dynamoDB.create_table(
        TableName='QualityAirTable',
        KeySchema=[{'AttributeName': 'id', 'KeyType': 'HASH'}],
        AttributeDefinitions=[{'AttributeName': 'id', 'AttributeType': 'S'}],
        ProvisionedThroughput={'ReadCapacityUnits': 1000, 'WriteCapacityUnits': 1000}
        )
        table.wait_until_exists()
        return table

We will define a function for setting the sqs queues 

In [4]:
def create_sqs_queue(SQS, queue_name):
    try:
        response = SQS.create_queue(QueueName=queue_name)
        print(f"Queue '{queue_name}' created successfully. URL: {response['QueueUrl']}")
        return response['QueueUrl']
    except Exception as e:
        print(f"Error creating SQS queue '{queue_name}': {e}")
        return None

We will define the function meant for the launching of the EC2 instances by handling the creation, waiting for it to become running, and providing details about the instance once it's ready

In [5]:
def create_instance(ec2, ami_id, instance_type, security_group_ids, key_name, user_data_script, instance_name):
    try:
        instances = ec2.run_instances(
            ImageId=ami_id,
            MinCount=1,
            MaxCount=1,
            InstanceType=instance_type,
            UserData=user_data_script,
            SecurityGroupIds=security_group_ids,
            KeyName=key_name,
            TagSpecifications=[
                {
                    'ResourceType': 'instance',
                    'Tags': [
                        {
                            'Key': 'Name',
                            'Value': instance_name,
                        },
                    ],
                },
            ],
        )['Instances']

        instance_id = instances[0]['InstanceId']
        print(f"Instance {instance_id} ({instance_name}) created, waiting for it to run...")
        
        ec2.get_waiter('instance_running').wait(InstanceIds=[instance_id])
        print(f"Instance {instance_id} ({instance_name}) is running.")
        instance_description = ec2.describe_instances(InstanceIds=[instance_id])
        time.sleep(60)
        return instance_description['Reservations'][0]['Instances'][0]
        
    except Exception as e:
        print(f"Error creating EC2 instance: {e}")
        return None

The next function serves as a utility for uploading our files to the instances we launched 

In [6]:
def upload_script(ec2, instance_id, script_local_path, key_path):
    instance_description = ec2.describe_instances(InstanceIds=[instance_id])
    DNS = instance_description['Reservations'][0]['Instances'][0]['PublicDnsName']

    scp_command = f"scp -o StrictHostKeyChecking=no -i {key_path} {script_local_path} ec2-user@{DNS}:/home/ec2-user/"
    scp_result = subprocess.run(scp_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    if scp_result.returncode != 0:
        print("SCP Error (stdout):", scp_result.stdout)
        print("SCP Error (stderr):", scp_result.stderr)
        return False
    time.sleep(5) 
    return True

The next function is meant for the remote execution of the programs we have sent to the instances 

In [7]:
def execute_script(ec2, instance_id, key_path, execution_file):
    instance_description = ec2.describe_instances(InstanceIds=[instance_id])
    DNS = instance_description['Reservations'][0]['Instances'][0]['PublicDnsName']

    # Execute the script immediately
    ssh_command = f"ssh -o StrictHostKeyChecking=no -i \"{key_path}\" ec2-user@{DNS} \"/usr/bin/python3 /home/ec2-user/{execution_file} > /home/ec2-user/execute.log 2>&1 &\""

    ssh_result = subprocess.run(ssh_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    if ssh_result.returncode != 0:
        print("Execution Error (stdout):", ssh_result.stdout)
        print("Execution Error (stderr):", ssh_result.stderr)
        return False
    
    return True

This function automates the setup and deployment of an application on an EC2 instance, including uploading the application, configuring the environment, and running the application as a background process

In [8]:
def setup_ClientApp(instance, app_zip_path, public_ip):
    try:
        key = paramiko.RSAKey.from_private_key_file('C:/Users/elkar/Downloads/labsuser.pem')
        client = paramiko.SSHClient()
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

        DNS = instance['PublicDnsName']
        client.connect(hostname=DNS, username='ec2-user', pkey=key)

        sftp = client.open_sftp()
        sftp.put(app_zip_path, '/home/ec2-user/ClientApp.zip')
        sftp.close()

        commands = [
            'sudo yum update -y',
            'sudo yum install python3 python3-pip unzip -y',
            'unzip /home/ec2-user/ClientApp.zip -d /home/ec2-user/app/',
            'pip3 install --user -r /home/ec2-user/app/ClientApp/requirements.txt'
        ]
        for command in commands:
            stdin, stdout, stderr = client.exec_command(command)
            print(stdout.read())

        print(f"Application deployed. Access it at http://{public_ip}:8501")   
        client.exec_command('nohup python3 /home/ec2-user/app/ClientApp/App.py > app.log 2>&1 &')
        time.sleep(5)

        client.close()
        return True  # Indicate successful completion
    except Exception as e:
        print(f"Error setting up ClientApp instance: {e}")
        return False

Now that we have defined the necessary libraries we will proceed to the main method of our class that It creates a distributed computing environment where the 'Master' instance manages tasks, the 'Worker' instances perform computations, and the 'ClientApp' serves an application

In [9]:
def main():
    ec2 = boto3.client('ec2', region_name='us-east-1')
    dynamoDB = boto3.resource('dynamodb', region_name='us-east-1')
    SQS = boto3.client('sqs', region_name='us-east-1')
    
    ami_id = 'ami-079db87dc4c10ac91'  # Replace with your AMI ID
    instance_type = 't2.micro'
    security_group_ids = ['sg-0e517e6668cedabd3']  # Replace with your security group ID
    key_name = 'vockey'  # Replace with your key name
    key_path = 'C:/Users/elkar/Downloads/labsuser.pem'  # Replace with your key path

    # User data scripts
    user_data_master = '''#!/bin/bash
                          sudo yum update -y
                          sudo yum install python3 python3-pip -y 
                          sudo yum install cronie -y
                          sudo service crond start
                          sudo chkconfig crond on
                          sudo pip3 install requests boto3
                          '''  # User data for 'Master' instance

    user_data_client_load = '''#!/bin/bash
                               sudo yum update -y
                               sudo yum install python3 python3-pip -y
                               sudo pip3 install requests boto3
                               '''  # User data for 'Test_Instance' instance

    # Create DynamoDB table
    create_DB(dynamoDB)

    # Create SQS queues
    request_send_queue_url = create_sqs_queue(SQS, 'RequestSend')
    request_receive_queue_url = create_sqs_queue(SQS, 'RequestReceive')

    # Launch and set up the 'Master' EC2 instance
    master_instance = create_instance(ec2, ami_id, instance_type, security_group_ids, key_name, user_data_master, 'Master')
    if master_instance:
        if upload_script(ec2, master_instance['InstanceId'], 'C:/Users/elkar/Downloads/430915_CC_Code/Master_Instance/master.py', key_path):
            if execute_script(ec2, master_instance['InstanceId'], key_path, 'master.py'):
                print("Script uploaded and executed on the 'Master' instance.")
            else:
                print("Failed to execute script on the 'Master' instance.")
        else:
            print("Failed to upload script to the 'Master' instance.")
    time.sleep(100)

    # Launch and set up the 'ClientApp' EC2 instance after 'Master' instance
    client_app_zip_path = 'C:/Users/elkar/Downloads/430915_CC_Code/ClientApp.zip'
    client_app_instance = create_instance(ec2, ami_id, instance_type, security_group_ids, key_name, '', 'ClientApp')
    if client_app_instance:
        public_ip = client_app_instance.get('PublicIpAddress')
        if setup_ClientApp(client_app_instance, client_app_zip_path, public_ip):
            print(f"'ClientApp' instance setup completed successfully.")
        else:
            raise Exception("Failed to set up 'ClientApp' instance.")
    else:
        raise Exception("Failed to create 'ClientApp' instance.")
     
    # Launch and set up the 'Worker' EC2 instance after 'Master' instance
    worker_instance = create_instance(ec2, ami_id, instance_type, security_group_ids, key_name, user_data_client_load, 'Worker')
    if worker_instance:
        if upload_script(ec2, worker_instance['InstanceId'], 'C:/Users/elkar/Downloads/430915_CC_Code/Worker_node/Worker.py', key_path):
            if execute_script(ec2, worker_instance['InstanceId'], key_path, 'Worker.py'):
                print("Script uploaded and executed on the 'Worker' instance .")
            else:
                print("Failed to execute script on the 'Worker' instance.")
        else:
            print("Failed to upload script to the 'Worker' instance.")    
        
    # Launch and set up the 'Test_Instance' EC2 instance
    client_load_instance = create_instance(ec2, ami_id, instance_type, security_group_ids, key_name, user_data_client_load, 'Test_Instance')
    if client_load_instance:
        if upload_script(ec2, client_load_instance['InstanceId'], 'C:/Users/elkar/Downloads/430915_CC_Code/Test_Instance/TestPerformance.py', key_path):
            if execute_script(ec2, client_load_instance['InstanceId'], key_path, 'TestPerformance.py'):
                print("Script uploaded and executed on the 'Test_Instance' instance.")
            else:
                print("Failed to execute script on the 'Test_Instance' instance.")
        else:
            print("Failed to upload script to the 'Test_Instance' instance.")

if __name__ == '__main__':
    main()
 

Queue 'RequestSend' created successfully. URL: https://sqs.us-east-1.amazonaws.com/513771522821/RequestSend
Queue 'RequestReceive' created successfully. URL: https://sqs.us-east-1.amazonaws.com/513771522821/RequestReceive
Instance i-0413074ba0dc2507e (Master) created, waiting for it to run...
Instance i-0413074ba0dc2507e (Master) is running.
Script uploaded and executed on the 'Master' instance.
Instance i-067af6687d7f4ccd4 (ClientApp) created, waiting for it to run...
Instance i-067af6687d7f4ccd4 (ClientApp) is running.
b'Last metadata expiration check: 0:00:55 ago on Sun Dec 31 02:00:50 2023.\nDependencies resolved.\nNothing to do.\nComplete!\n'
b'Last metadata expiration check: 0:00:56 ago on Sun Dec 31 02:00:50 2023.\nPackage python3-3.9.16-1.amzn2023.0.6.x86_64 is already installed.\nPackage unzip-6.0-57.amzn2023.0.2.x86_64 is already installed.\nDependencies resolved.\n================================================================================\n Package             Arch     